In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.enable_eager_execution()

### Test eager execution

In [3]:
tf.executing_eagerly()

True

In [4]:
x = [[2],[1]]
m = tf.matmul(tf.transpose(x),x)
print(m)

tf.Tensor([[5]], shape=(1, 1), dtype=int32)


### Model Architecture 

In [6]:
import os
#os.chdir("../src/models/")
import atom
from atom import interatomic_distances
from atom import site_rdf

In [10]:
### read tfrecord ###
def read_tfrecord(tfrecord_file, train_position, batch_size = 100, num_epochs = None, shuffle = False, buffer_size = 99000,cutoff = 20, step=0.2):
    def parser(record):
        featnames = ['elements'] + [train_position] + ['targets']
        feattypes = [tf.int64, tf.float32, tf.float32]
        features = { featname :tf.FixedLenFeature([], tf.string) for featname in featnames}
        tfrecord_features = tf.parse_single_example(record, features=features, name='features')
        newfeatures = {}
        for featname, feattype in zip(featnames, feattypes):
            feat = tf.decode_raw(tfrecord_features[featname], feattype)
            if featname == 'positions' or featname == 'mmffpositions' or featname == "positions1" or featname == "positions2":
                feat = tf.reshape(feat, (-1, 3))
            if featname == 'targets':
                feat = tf.reshape(feat, (15, 1))
            newfeatures[featname] = feat
        return newfeatures
    df = tf.data.TFRecordDataset(tfrecord_file)
    df = df.map(parser)
    if shuffle != False:
        df = df.shuffle(buffer_size = buffer_size)
    df = df.repeat(num_epochs)
    df = df.padded_batch(batch_size= batch_size,padded_shapes={"elements":[None,],train_position:[None,3],"targets":[15,1]},padding_values=None)
    iterator = df.make_one_shot_iterator()
    features = iterator.get_next()
    
    distances = interatomic_distances(tf.cast(features[train_position], tf.float32))
    rdf = site_rdf(distances, cutoff=cutoff, step=step, width=1)

    features["rdf"] = rdf
    return iterator, features

One thing should be noted is that when enable eager, tf.data.Dataset.make_initializable_iterator can't be used <br>
Use make_one_shot_iterator instead, which generate a iterator with all information of dataset<br>
Use iterator.get_next() can get the first element in iterator<br>
If you want to go through all of the data, just use for loop and contain get_next in loop <br>
With no eager on, don't need to contain get_next in loop <br>

In [8]:
### example ###
dataset = tf.data.Dataset.range(100)
iterator = dataset.make_one_shot_iterator()
### Use for loop to go through all instances, remember contain get_next in loop ###
for i in range(10):
    next_element = iterator.get_next()
    print(next_element)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [11]:
tfrecord_file = "../../data/external/qm9mmff/train_new.tfrecord"
train_position = "positions"
### now train_features is only the first instance of iterator
iterator, train_features = read_tfrecord(tfrecord_file, train_position, batch_size = 10, num_epochs = 1, shuffle = False, buffer_size = 99000,cutoff = 3, step=0.1)

In [234]:
train_features["elements"]
### rdf is the distance input after Gaussian expansion with cutoff 3A and 0.1 step ###
train_features["rdf"][0][0][0:2]

<tf.Tensor: id=80, shape=(10, 25), dtype=int64, numpy=
array([[6, 6, 6, 6, 6, 7, 6, 7, 8, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0],
       [6, 6, 6, 6, 6, 8, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [6, 6, 6, 6, 7, 6, 8, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0],
       [8, 6, 6, 8, 6, 6, 8, 6, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [6, 8, 6, 6, 8, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0],
       [6, 6, 6, 6, 6, 7, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0],
       [6, 6, 6, 6, 6, 7, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0],
       [6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1],
       [6, 7, 6, 6, 6, 8, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0],
       [6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0]])>

<tf.Tensor: id=2111, shape=(2, 30), dtype=float32, numpy=
array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.19509683e-36, 4.20253336e-31,
        5.06823009e-26, 2.09623642e-21, 2.97345457e-17, 1.44650622e-13,
        2.41332371e-10, 1.38086264e-07, 2.70971286e-05, 1.82361354e-03,
        4.20901105e-02, 3.33169371e-01, 9.04456973e-01, 8.42070758e-01,
        2.68872917e-01, 2.94430852e-02, 1.10575103e-03, 1.42419467e-05,
        6.29094359e-08, 9.53030432e-1

In [42]:
### Hyperparameters ###
n_basis = 256
n_factors = 60
n_interactions = 7

In [13]:
Z = train_features["elements"]
C = train_features["rdf"]

In [14]:
print("Input charge vetor shape:")
print(Z.get_shape().as_list())
print("Input distance vector shape:")
print(C.get_shape().as_list())

Input charge vetor shape:
[10, 25]
Input distance vector shape:
[10, 25, 25, 30]


In [235]:
### Set mask ###
mask = tf.cast(tf.expand_dims(Z, 1) * tf.expand_dims(Z, 2), tf.float32)
mask = tf.matrix_set_diag(mask, tf.zeros_like(tf.matrix_diag_part(mask)))
mask = tf.expand_dims(mask, -1)
print(mask[0][0])

tf.Tensor(
[[ 0.]
 [36.]
 [36.]
 [36.]
 [36.]
 [42.]
 [36.]
 [42.]
 [48.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]], shape=(25, 1), dtype=float32)


This mask will be used to time the interaction term, thus only neighbor atom (not itself or 0) should be considered

In [150]:
### Define fully connected layers, and useful functions ###
def shape(x):
    if isinstance(x, tf.Tensor):
        return x.get_shape().as_list()
    return np.shape(x)

def glorot_uniform(shape, dtype, partition_info=None):
    if not dtype.is_floating:
        raise ValueError("Expected floating point type, got %s." % dtype)

    n_in = np.prod(shape[:-1])
    n_out = shape[-1]

    r = tf.cast(tf.sqrt(6. / (n_in + n_out)), tf.float32)
    return tf.random_uniform(shape, -r, r, dtype=dtype)

def dense(x, n_out,nonlinearity=None,use_bias=True,
          weight_init=glorot_uniform,bias_init=tf.constant_initializer(0.),trainable=True,
          scope=None, reuse=False, name='Dense'):
    x_shape = shape(x)
    ndims = len(x_shape)
    n_in = x_shape[-1]
    with tf.variable_scope(scope, default_name=name, values=[x],
                           reuse=reuse) as scope:
        # reshape for broadcasting
        xr = tf.reshape(x, (-1, n_in))
        W = tf.get_variable('W', shape=(n_in, n_out),
                            initializer=weight_init,
                            trainable=trainable)
        tf.add_to_collection(tf.GraphKeys.WEIGHTS, W)
        tf.summary.histogram('W', W)

        y = tf.matmul(xr, W)

        if use_bias:
            b = tf.get_variable('b', shape=(n_out,),
                                initializer=bias_init,
                                trainable=trainable)
            tf.add_to_collection(tf.GraphKeys.BIASES, b)
            tf.summary.histogram('b', b)
            y += b

        if nonlinearity:
            y = nonlinearity(y)

        new_shape = tf.concat([tf.shape(x)[:ndims - 1], [n_out]], axis=0)
        y = tf.reshape(y, new_shape)

        tf.summary.histogram('activations', y)
    return y

In [166]:
### Initilization of charge vector ###
### generate one hot encoding for input charge vector ###
I = np.eye(20).astype(np.float32)
ZZ = tf.nn.embedding_lookup(I, Z)
r = tf.sqrt(1. / tf.sqrt(float(n_basis)))
X = dense(ZZ, n_basis, use_bias=False, weight_init=tf.random_normal_initializer(stddev=r),scope = "initial_embedding")

In [162]:
print("Initial embedding shape: No bias, weight_init = tf.random_normal_initializer(stddev=r)")
print(X.get_shape())

Initial embedding shape: No bias, weight_init = tf.random_normal_initializer(stddev=r)
(10, 25, 256)


In [167]:
### Initial embedding weights ###
tf.get_collection(tf.GraphKeys.WEIGHTS, "initial_embedding")
tf.get_collection(tf.GraphKeys.BIASES, "initial_embedding")

[<tf.Variable 'initial_embedding/W:0' shape=(20, 256) dtype=float32, numpy=
 array([[ 0.11875952, -0.23058635,  0.08487264, ..., -0.04433521,
          0.24136327,  0.07745977],
        [ 0.00947792,  0.24367547, -0.55862415, ..., -0.2738677 ,
         -0.36462742, -0.31353384],
        [-0.3552743 , -0.20566714, -0.11362048, ...,  0.06695532,
          0.10689724, -0.22511972],
        ...,
        [-0.12171436, -0.00504464,  0.0300582 , ...,  0.01375662,
         -0.19546737, -0.1701538 ],
        [-0.3456712 , -0.53926086,  0.25564918, ...,  0.16367278,
          0.5340095 , -0.08660128],
        [-0.3445733 , -0.0302835 , -0.25506225, ...,  0.19609487,
         -0.6742876 ,  0.4401804 ]], dtype=float32)>]

[]

In [168]:
### Initilization of distance vector ###
fC = dense(C, n_factors, use_bias=True,scope = "initial_distance")

In [169]:
print("Initial distance shape: bias=True,weight_init=glorot_uniform,bias_init=tf.constant_initializer(0.)")
print(fC.get_shape())

Initial distance shape: bias=True,weight_init=glorot_uniform,bias_init=tf.constant_initializer(0.)
(10, 25, 25, 60)


In [170]:
### Initial distance weights ###
tf.get_collection(tf.GraphKeys.WEIGHTS, "initial_distance")
tf.get_collection(tf.GraphKeys.BIASES, "initial_distance")

[<tf.Variable 'initial_distance/W:0' shape=(30, 60) dtype=float32, numpy=
 array([[-0.22729053, -0.10617097,  0.21834058, ..., -0.14167622,
          0.1729415 ,  0.21726528],
        [ 0.12840334, -0.234496  , -0.25388485, ..., -0.01537693,
         -0.13276678,  0.10266411],
        [-0.19654092, -0.13293108, -0.00402111, ..., -0.2257106 ,
          0.03505188,  0.14333695],
        ...,
        [-0.17199841,  0.03974071, -0.1038437 , ...,  0.07267618,
          0.17445144,  0.25768486],
        [-0.25053346, -0.21315396,  0.22920305, ..., -0.05761026,
         -0.19399254,  0.11422804],
        [-0.05319934,  0.14005   , -0.11742249, ...,  0.11760095,
         -0.05191755,  0.19052589]], dtype=float32)>]

[<tf.Variable 'initial_distance/b:0' shape=(60,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [193]:
### Interaction Blocks ###
### Useful functions ###
def _softplus(x):
    return tf.log1p(tf.exp(x))

def shifted_softplus(x):
    """
    Softplus nonlinearity shifted by -log(2) such that shifted_softplus(0.) = 0.

    y = log(0.5e^x + 0.5)
    
    14 is the cutoff for exposion (if element in x > 14, it will be replaces by 0)
    
    _softplus(tf.where(x < 14., x, tf.zeros_like(x))) almost always less than 14

    """
    y = tf.where(x < 14., _softplus(tf.where(x < 14., x, tf.zeros_like(x))), x)
    return y - tf.log(2.)
def masked_reduce(x, mask=None, axes=None,
                  reduce_op=tf.reduce_sum,
                  keep_dims=False,
                  scope=None, name='masked_reduce'):
    scope_vars = [x]
    if mask is not None:
        scope_vars.append(mask)

    with tf.variable_scope(scope, default_name=name,
                           values=scope_vars) as scope:
        if mask is not None:
            mask = tf.cast(mask > 0, tf.float32)
            x *= mask

        y = reduce_op(x, axes, keep_dims)

    return y

def masked_sum(x, mask=None, axes=None,
               keep_dims=False,
               scope=None, name='masked_sum'):
    return masked_reduce(x, mask, axes, tf.reduce_sum,
                         keep_dims, scope, name)


def masked_mean(x, mask=None, axes=None,
                keep_dims=False,
                scope=None, name='masked_mean'):
    if mask is None:
        mred = masked_reduce(x, mask, axes, tf.reduce_mean,
                             keep_dims, scope, name)
    else:
        msum = masked_reduce(x, mask, axes, tf.reduce_sum,
                             keep_dims, scope, name)
        mask = tf.cast(mask > 0, tf.float32)
        N = tf.reduce_sum(mask, axes, keep_dims)
        N = tf.maximum(N, 1)
        mred = msum / N
    return mred


In [194]:
reuse = None
for i in range(n_interactions):
    ### increase initial embedding dimension ###
    tmp = tf.expand_dims(X, 1)
    ### one atom-wise layer ###
    fX = dense(tmp, n_factors, use_bias=True, scope='in2fac' + str(i), reuse=reuse)
    ### get interaction term
    fVj = fX * fC
    Vj = dense(fVj, n_basis, use_bias=False, weight_init=tf.constant_initializer(0.0),
               scope='fac2out' + str(i), reuse=reuse)
    Vj = shifted_softplus(Vj)
    ### use mask to remove non neighbor atoms and sum ###
    V = masked_sum(Vj, mask, axes=2)
    ### update atom embedding ###
    X += V


In [204]:
print("Final atom embedding for first molecule")
print(X[0])

Final atom embedding for first molecule
tf.Tensor(
[[-0.10555546  0.14823608 -0.18198036 ... -0.04932855 -0.17273639
  -0.13344556]
 [-0.10555546  0.14823608 -0.18198036 ... -0.04932855 -0.17273639
  -0.13344556]
 [-0.10555546  0.14823608 -0.18198036 ... -0.04932855 -0.17273639
  -0.13344556]
 ...
 [ 0.11875952 -0.23058635  0.08487264 ... -0.04433521  0.24136327
   0.07745977]
 [ 0.11875952 -0.23058635  0.08487264 ... -0.04433521  0.24136327
   0.07745977]
 [ 0.11875952 -0.23058635  0.08487264 ... -0.04433521  0.24136327
   0.07745977]], shape=(25, 256), dtype=float32)


In [208]:
print("Vj dimension:")
print(Vj.get_shape())
print("V dimension: neighbour effects has been summed for each atom")
print(V.get_shape())

Vj dimension:
(10, 25, 25, 256)
V dimension: neighbour effects has been summed for each atom
(10, 25, 256)


Since this is just a showcase and there is no training process, Vj is a zero matrix since weight_init using constant zero. However, with training process continue, weight in scope fac2out will be trained and Vj will be generated and X will be updated.

In [210]:
### Output Layer ###
o1 = dense(X, n_basis // 2, nonlinearity=tf.nn.tanh)
yi = dense(o1, 1, weight_init=tf.constant_initializer(0.0), use_bias=True)

In [212]:
print("Final atomic energy")
print(yi.get_shape())

Final atomic energy
(10, 25, 1)


In [214]:
### Normalization ###
### std, mu from training set ###
mu = -4.24368628211098
std = 0.18926335325096477
yi = yi * std + mu

In [221]:
### Add Atom Reference ###
### atom reference from QM calculation ###
atom_ref = np.load("../../data/raw/atomrefs.txt.npz")
atom_ref = atom_ref["atom_ref"][:,1:2]
atom_ref = tf.constant(atom_ref[0:10],tf.float32)
if atom_ref != 0.000:
    E0i = tf.nn.embedding_lookup(atom_ref, Z)
    yi += E0i

In [222]:
### Molecular Energy ###
### sum all molecules (non padded one) ###
atom_mask = tf.expand_dims(Z, -1)
mask = tf.cast(atom_mask > 0, tf.float32)
y = tf.reduce_sum(yi * mask, 1)

In [224]:
print("Final energy:")
print(y.get_shape())
print(y)

Final energy:
(10, 1)
tf.Tensor(
[[-11373.443]
 [ -9428.457]
 [-10935.859]
 [-11453.953]
 [-11984.322]
 [ -9994.539]
 [ -9901.754]
 [ -9592.669]
 [-11948.607]
 [ -8558.563]], shape=(10, 1), dtype=float32)


In [231]:
print("target:")
print(train_features["targets"][:,10])

target:
tf.Tensor(
[[-11376.758]
 [ -9429.651]
 [-10937.167]
 [-11453.006]
 [-11980.546]
 [ -9990.978]
 [ -9899.319]
 [ -9589.001]
 [-11945.698]
 [ -8551.802]], shape=(10, 1), dtype=float32)


Without training, the error is very large